In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Valuation with recipes

This notebook shows how to value a portfolio using recipes with different pricing sources

Attributes
----------
valuation
transactions
recipes
manifests
"""

toggle_code("Hide docstring")

# Valuation with recipes

This notebooks shows how to value a portfolio using recipes with different pricing sources

## Setup LUSID

In [2]:
# Import system packages

# Import lusid specific packages
# These are the core lusid packages for interacting with the API via Python
import lusid
import lusid.models as models
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
)
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

import os
import pandas as pd
import pytz
from datetime import datetime

pd.set_option("display.max_columns", None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

In [3]:
# define the apis
aggregation_api = api_factory.build(lusid.AggregationApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

## Load source data

Create a scope and portfolio code

In [4]:
scope = "valuation-sample"
portfolio_code = "EQUITY_UK"

In [5]:
df = pd.read_csv("data/recipe-valuation/equity_transactions.csv")
df

,portfolio_code,portfolio_name,portfolio_base_currency,ISIN,sedol,instrument_type,instrument_id,name,txn_id,txn_type,txn_trade_date,txn_settle_date,txn_units,txn_price,txn_consideration,currency,strategy,cash_transactions
0,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0002162385,0216238,equity,EQ_1234,Aviva,txn-1,StockIn,02/01/2020,04/01/2020,120000,4.23,600000,GBP,ftse_tracker,NaN
1,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00BH0P3Z91,BH0P3Z9,equity,EQ_1235,BHP,txn-2,StockIn,02/01/2020,04/01/2020,60000,17.89,1080000,GBP,ftse_tracker,NaN
2,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0031348658,3134865,equity,EQ_1236,Barclays,txn-3,StockIn,02/01/2020,04/01/2020,150000,1.80,300000,GBP,ftse_tracker,NaN
3,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0007980591,0798059,equity,EQ_1237,BP,txn-4,StockIn,02/01/2020,04/01/2020,100000,4.75,500000,GBP,ftse_tracker,NaN
4,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0005405286,0540528,equity,EQ_1238,HSBC,txn-5,StockIn,02/01/2020,04/01/2020,20000,5.89,120000,GBP,ftse_tracker,NaN
5,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0006043169,0604316,equity,EQ_1239,Morrisons,txn-6,StockIn,15/01/2020,17/01/2020,180000,1.87,360000,GBP,ftse_tracker,NaN
6,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB0008847096,0884709,equity,EQ_1240,Tesco,txn-7,StockIn,16/01/2020,18/01/2020,4000,2.47,36000,GBP,ftse_tracker,NaN
7,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00BGDT3G23,BGDT3G2,equity,EQ_1241,Rightmove,txn-8,StockIn,15/01/2020,17/01/2020,80000,6.59,480000,GBP,ftse_tracker,NaN
8,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00BH4HKS39,BH4HKS3,equity,EQ_1242,vodafone,txn-9,StockIn,15/01/2020,17/01/2020,450000,1.56,450000,GBP,ftse_tracker,NaN
9,EQUITY_UK,LUSID's top 10 FTSE stock portfolio,GBP,GB00B1XZS820,B1XZS82,equity,EQ_1243,Anglo American plc,txn-10,StockIn,15/01/2020,17/01/2020,35000,21.68,700000,GBP,ftse_tracker,NaN


## Load instruments

Create the instruments and add on ISIN and Sedol as identifiers

In [6]:
instrument_mapping = {
    "identifier_mapping": {
        "ClientInternal": "instrument_id",
        "Isin": "ISIN",
        "Sedol": "sedol",
    },
    "required": {"name": "name"},
}

In [7]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=instrument_mapping["required"],
    mapping_optional={},
    file_type="instruments",
    identifier_mapping=instrument_mapping["identifier_mapping"],
)

succ, failed, errors = format_instruments_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,10,0,0


## Create portfolio

In [8]:
portfolio_mapping = {
    "required": {
        "code": "portfolio_code",
        "display_name": "portfolio_name",
        "base_currency": "$GBP",
    },
    "optional": {"created": "$2020-01-01T00:00:00+00:00"},
}

In [9]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=portfolio_mapping["required"],
    mapping_optional=portfolio_mapping["optional"],
    file_type="portfolios",
    sub_holding_keys=[],
)

succ, failed = format_portfolios_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,1,0,0


## Load in transactions

In [10]:
transaction_mapping = {
    "identifier_mapping": {"ClientInternal": "instrument_id",},
    "required": {
        "code": "portfolio_code",
        "transaction_id": "txn_id",
        "type": "txn_type",
        "transaction_price.price": "txn_price",
        "transaction_price.type": "$Price",
        "total_consideration.amount": "txn_consideration",
        "units": "txn_units",
        "transaction_date": "txn_trade_date",
        "total_consideration.currency": "portfolio_base_currency",
        "settlement_date": "txn_settle_date",
    },
    "optional": {},
    "properties": [],
}

In [11]:
result = load_from_data_frame(
    api_factory=api_factory,
    scope=scope,
    data_frame=df,
    mapping_required=transaction_mapping["required"],
    mapping_optional=transaction_mapping["optional"],
    file_type="transactions",
    identifier_mapping=transaction_mapping["identifier_mapping"],
    property_columns=transaction_mapping["properties"],
    properties_scope=scope,
)

succ, failed = format_transactions_response(result)
pd.DataFrame(
    data=[{"success": len(succ), "failed": len(failed), "errors": len(errors)}]
)

,success,failed,errors
0,1,0,0


## Load in quotes

Load the source quotes containing bid, mid and ask

In [12]:
quotes_df = pd.read_csv("data/recipe-valuation/equity_quotes.csv")
quotes_df

,ISIN,quote_date,bid,mid,ask
0,GB0002162385,24-Apr-20,2.29,2.30,2.31
1,GB00BH0P3Z91,24-Apr-20,12.81,12.87,12.93
2,GB0031348658,24-Apr-20,0.88,1.88,0.88
3,GB0007980591,24-Apr-20,3.06,3.08,3.10
4,GB0005405286,24-Apr-20,4.00,4.02,4.04
5,GB0006043169,24-Apr-20,1.87,1.88,1.89
6,GB0008847096,24-Apr-20,2.35,2.36,2.37
7,GB00BGDT3G23,24-Apr-20,4.63,4.65,4.67
8,GB00BH4HKS39,24-Apr-20,1.08,1.09,1.10
9,GB00B1XZS820,24-Apr-20,13.90,13.97,14.04


In [13]:
price_fields = ["bid", "mid", "ask"]

quotes_mapping = {
    "quote_id.quote_series_id.instrument_id_type": "$Isin",
    "quote_id.effective_at": "quote_date",
    "quote_id.quote_series_id.provider": "$Lusid",
    "quote_id.quote_series_id.quote_type": "$Price",
    "quote_id.quote_series_id.instrument_id": "ISIN",
    "metric_value.unit": "$GBP",
}

In [14]:
for price_field in price_fields:

    quotes_mapping["quote_id.quote_series_id.field"] = f"${price_field}"
    quotes_mapping["metric_value.value"] = price_field

    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=quotes_df,
        mapping_required=quotes_mapping,
        mapping_optional={},
        file_type="quotes",
    )

    succ, failed, errors = format_quotes_response(result)
    display(
        pd.DataFrame(
            data=[
                {
                    f"[{price_field}] success": len(succ),
                    "failed": len(failed),
                    "errors": len(errors),
                }
            ]
        )
    )

,[bid] success,failed,errors
0,10,0,0


,[mid] success,failed,errors
0,10,0,0


,[ask] success,failed,errors
0,10,0,0


## Valuation

Perform a valuation on the portfolio

In [15]:
def aggregation_request(valuation_effectiveAt, price_field):
    inline_recipe = models.ConfigurationRecipe(
        scope="User",
        code="valuation_recipe",
        market=models.MarketContext(
            market_rules=[
                # define how to resolve the quotes
                models.MarketDataKeyRule(
                    key="Quote.Isin.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field=price_field,
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="Isin",
                default_scope=scope,
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

    return models.AggregationRequest(
        inline_recipe=inline_recipe,
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Holding/default/PV", "Proportion"),
            models.AggregateSpec("Holding/default/PV", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        # choose the valuation time for the request
        effective_at=valuation_effectiveAt,
    )

In [16]:
# Create recipes
for price_field in ["mid", "bid", "ask"]:

    # Create a recipe to perform a valuation
    configuration_recipe = models.ConfigurationRecipe(
        scope="User",
        code="valuation_recipe" + "_" + price_field,
        market=models.MarketContext(
            market_rules=[
                # define how to resolve the quotes
                models.MarketDataKeyRule(
                    key="Quote.Isin.*",
                    supplier="Lusid",
                    data_scope=scope,
                    quote_type="Price",
                    field=price_field,
                ),
            ],
            options=models.MarketOptions(
                default_supplier="Lusid",
                default_instrument_code_type="Isin",
                default_scope=scope,
            ),
        ),
        pricing=models.PricingContext(
            options={"AllowPartiallySuccessfulEvaluation": True},
        ),
    )

    upsert_configuration_recipe_response = configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )

In [17]:
def generate_valuation_request(valuation_effectiveAt, price_field):

    # Create the valuation request
    valuation_request = models.ValuationRequest(
        recipe_id=models.ResourceId(
            scope="User", code="valuation_recipe" + "_" + price_field
        ),
        metrics=[
            models.AggregateSpec("Instrument/default/Name", "Value"),
            models.AggregateSpec("Holding/default/PV", "Proportion"),
            models.AggregateSpec("Holding/default/PV", "Sum"),
            models.AggregateSpec("Holding/default/Units", "Sum"),
            models.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        portfolio_entity_ids=[
            models.PortfolioEntityId(scope=scope, code=portfolio_code)
        ],
        valuation_schedule=models.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

Run a valuation using a recipe using "mid" quotes

In [18]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime(year=2020, month=4, day=24, tzinfo=pytz.UTC), "mid"
    )
)

pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/PV),Sum(Holding/default/Units),Holding/default/Error
0,Aviva,0.080752,276000.0,120000.0,
1,BHP,0.225929,772200.0,60000.0,
2,Barclays,0.082507,282000.0,150000.0,
3,BP,0.090114,308000.0,100000.0,
4,HSBC,0.023523,80400.0,20000.0,
5,Morrisons,0.099008,338400.0,180000.0,
6,Rightmove,0.108839,372000.0,80000.0,
7,vodafone,0.143510,490500.0,450000.0,
8,Anglo American plc,0.143056,488950.0,35000.0,
9,Tesco,0.002762,9440.0,4000.0,


Run a valuation using a recipe using "bid" quotes

In [19]:
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime(year=2020, month=4, day=24, tzinfo=pytz.UTC), "bid"
    )
)

pd.DataFrame(aggregation.data)

,Instrument/default/Name,Proportion(Holding/default/PV),Sum(Holding/default/PV),Sum(Holding/default/Units),Holding/default/Error
0,Aviva,0.084546,274800.0,120000.0,
1,BHP,0.236470,768600.0,60000.0,
2,Barclays,0.040612,132000.0,150000.0,
3,BP,0.094145,306000.0,100000.0,
4,HSBC,0.024613,80000.0,20000.0,
5,Morrisons,0.103560,336600.0,180000.0,
6,Rightmove,0.113959,370400.0,80000.0,
7,vodafone,0.149525,486000.0,450000.0,
8,Anglo American plc,0.149678,486500.0,35000.0,
9,Tesco,0.002892,9400.0,4000.0,
